<a href="https://colab.research.google.com/github/Xornotor/PPGEEC-CompEvolutiva-Atividades/blob/main/Avalia%C3%A7%C3%A3o_1A_Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# **Avaliação 1A - Computação Evolutiva e Meta-heurísticas**

**Docente:** Prof. Dr. Edmar Egídio Purcino de Souza

**Discentes:** André Paiva, Gabriel Lucas, Márcio Barros e Shaísta Câmara

## 1 - Importação de Dependências e Parametrização

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
from plotly import graph_objects as go
from itertools import combinations
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
# Parâmetros do algoritmo genético

#------------------------------------------------------------------
# Escolha a função de fitness a ser usada: "dropwave" ou "levi"
FUNCTION_NAME = "dropwave"
#FUNCTION_NAME = "levi"
#------------------------------------------------------------------

# Quantidade de indivíduos por geração
POPULATION_SIZE = 100

# Quantidade máxima de gerações
GENERATION_COUNT = 100

# Quantidade de indivíduos a ser preservados no elitismo
ELITISM = 10

# Seleção: 1 = Roleta, 2 = Torneio, 3 = Aleatório (50/50)
SELECTION_TYPE = 2

# Recombinação: 1 = Aritmética, 2 = Discreta, 3 = Aleatório (50/50)
CROSSOVER_TYPE = 3

# Taxa de recombinação
CROSSOVER_RATE = 0.8

# Taxa de mutação
MUTATION_RATE = 0.5

# Aumento da taxa de mutação em caso de baixa diversidade
MUTATION_ENHANCEMENT = True
MUTATION_ENHANCE_SETPOINT = 5
MUTATION_RATE_HIGH = 0.8

# Reinicialização da população em caso de baixa diversidade
POPULATION_RESET = True
POPULATION_RESET_SETPOINT = 1

# Parada antecipada em caso de convergência
EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 10

# Quantidade de genes por cromossomo
CHROMOSOME_LENGTH = 2

# Limites de valores para os genes
LOWER_BOUND = -100
UPPER_BOUND = 100

## 2 - Definição de função objetivo (*fitness*)

In [ ]:
if FUNCTION_NAME == "dropwave":
    def fitness(chromosome):
        # Dropwave
        x1 = chromosome[0]
        x2 = chromosome[1]
        r = np.sqrt(x1**2 + x2**2)
        numerator = -(1 + np.cos(12 * r))
        denominator = 0.5 * (x1**2 + x2**2) + 2
        return numerator / denominator
elif FUNCTION_NAME == "levi":
    def fitness(chromosome):
        # Levi n. 13
        x1 = chromosome[0]
        x2 = chromosome[1]
        term1 = np.sin(3 * np.pi * x1)**2
        term2 = (x1 - 1)**2 * (1 + np.sin(3 * np.pi * x2)**2)
        term3 = (x2 - 1)**2 * (1 + np.sin(2 * np.pi * x2)**2)
        return term1 + term2 + term3
else:
    raise ValueError("Função de fitness desconhecida.")

In [ ]:
# Plot interativo da função objetivo
def fitness_plot():
    if FUNCTION_NAME == "dropwave":
        x = y = np.linspace(-5, 5, 500)
    elif FUNCTION_NAME == "levi":
        x = y = np.linspace(-10, 10, 500)

    chromosome = np.meshgrid(x, y)
    Z = fitness(chromosome)

    fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='rainbow')])
    fig.update_layout(title=f"Função {FUNCTION_NAME.capitalize()}",
                    autosize=False, width=800, height=600,
                    margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

## 3 - Funções auxiliares para o algoritmo genético

In [ ]:
# Instanciação de população inicial
def generate_population():
    population = [(np.random.uniform(LOWER_BOUND, UPPER_BOUND, CHROMOSOME_LENGTH), 0) for _ in range(POPULATION_SIZE)]
    population = [(np.array(ind), fitness(ind)) for ind, _ in population]
    return population

# Medidas de diversidade
def diversity_metrics(population):
    genes = np.array([chromosome for chromosome, _ in population])
    std_devs = np.std(genes, axis=0)
    mean_distance_to_center = np.mean(np.linalg.norm(genes - np.mean(genes, axis=0), axis=1))
    distances = [np.linalg.norm(a - b) for a, b in combinations(genes, 2)]
    mean_pairwise_distance = np.mean(distances)
    return std_devs, mean_distance_to_center, mean_pairwise_distance

# Seleção
def selection(population):
    selected_parents = []
    random_sel = np.random.uniform(0, 1) # Seleção aleatória: > 0.5 Torneio, c.c. Roleta

    if(SELECTION_TYPE == 1 or (SELECTION_TYPE == 3 and random_sel <= 0.5)):
        pass # [IMPLEMENTAR ROLETA]
    elif(SELECTION_TYPE == 2 or (SELECTION_TYPE == 3 and random_sel > 0.5)):
        tournament_size = 10
        for _ in range(len(population)):
            tournament = random.sample(population, tournament_size-1)
            winner = min(tournament, key=lambda x: x[1]) # x[1] é o fitness na populatin/tourment
            selected_parents.append(winner[0]) # winner[0] pega somente os valores dos genes (x1,x2)
    else:
        raise ValueError("SELECTION_TYPE inválido.")

    best_individual = min(population, key=lambda x: x[1])[0]
    selected_parents.append(best_individual)
    return selected_parents

# Recombinação
def crossover(parent1, parent2):
    if np.random.uniform(0, 1) < CROSSOVER_RATE:
        random_cross = np.random.uniform(0, 1) # Seleção aleatória: > 0.5 Discreta, c.c. Aritmética
        if(CROSSOVER_TYPE == 1 or (CROSSOVER_TYPE == 3 and random_cross <= 0.5)):
            interp = np.random.uniform(0, 1)
            child1 = (interp * parent1) + ((1 - interp) * parent2)
            child2 = (interp * parent2) + ((1 - interp) * parent1)
            return child1, child2
        elif(CROSSOVER_TYPE == 2 or (CROSSOVER_TYPE == 3 and random_cross > 0.5)):
            crossover_point = random.randint(1, CHROMOSOME_LENGTH - 1)
            child1 = np.append(parent1[:crossover_point], parent2[crossover_point:])
            child2 = np.append(parent2[:crossover_point], parent1[crossover_point:])
            return child1, child2
        else:
            raise ValueError("CROSSOVER_TYPE inválido.")
    else:
        return parent1, parent2

# Mutação
def mutate(individual, mut_rate=MUTATION_RATE):
    mutated_individual = individual.copy()
    for i in range(CHROMOSOME_LENGTH):
        if random.random() < mut_rate:
            mutated_individual[i] += random.uniform(-1, 1)
    return np.clip(mutated_individual, LOWER_BOUND, UPPER_BOUND)

# Plot das medidas de diversidade
def plot_diversity_metrics(std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list):
    generations = range(len(std_devs_list))
    std_x1 = [std[0] for std in std_devs_list]
    std_x2 = [std[1] for std in std_devs_list]

    plt.figure(figsize=(10, 3))

    # Gráfico 1: Desvio padrão dos genes
    plt.subplot(1, 3, 1)
    plt.plot(generations, std_x1, label='Std x1')
    plt.plot(generations, std_x2, label='Std x2')
    plt.title('Desvio Padrão dos Genes')
    plt.xlabel('Geração')
    plt.ylabel('Desvio Padrão')
    plt.legend()
    plt.grid(True)

    # Gráfico 2: Distância ao centro
    plt.subplot(1, 3, 2)
    plt.plot(generations, mean_distance_to_center_list, color='orange')
    plt.title('Distância Média ao Centro')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    # Gráfico 3: Distância média entre pares
    plt.subplot(1, 3, 3)
    plt.plot(generations, mean_pairwise_distance_list, color='green')
    plt.title('Distância Média entre Indivíduos')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

## 4 - Algoritmo Genético

In [ ]:
def genetic_algorithm(iter_print=False):
    # Inicialização da primeira geração
    population = generate_population()

    # Lista de melhores valores de cada geração
    bestValue = []

    # Variáveis de controle para early stopping
    last_iter_with_enhancement = 0
    fitness_for_last_iter_with_enhance = np.inf

    # Listas de medidas de diversidade
    std_devs_list = []
    mean_distance_to_center_list = []
    mean_pairwise_distance_list = []

    # Taxa de mutação (pode aumentar em caso de baixa diversidade)
    mut_rate = MUTATION_RATE

    for iter in range(GENERATION_COUNT):
        # Avalia a população com a função de fitness escolhida
        population = [(chromosome, fitness(chromosome)) for chromosome, _ in population]

        # Seleciona os pais
        parents = selection(population) # retorna um vetor de size population+1, isso é importante pro loop seguinte funcionar, como foi feito

        # Realiza o crossover e a mutação para gerar descendentes
        offspring = []
        # Vai de 2 em 2 dentro da população
        for i in range(0, POPULATION_SIZE, 2):
            parent1 = np.array(parents[i])
            parent2 = np.array(parents[i + 1])
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mut_rate)
            child2 = mutate(child2, mut_rate)
            offspring.extend([child1, child2])

        # Recria a população a partir dos descendentes (sem avaliar fitness)
        population_x = [(chromosome, 0) for chromosome in offspring]

        # Elitismo: mantém os melhores indivíduos da geração anterior
        elites = sorted(population, key=lambda x: x[1])[:ELITISM]

        best_individual = min(population, key=lambda x: x[1])[0] # O [0] no final é para pegar apenas o cromossomo, depois de avaliar o menor fitness da população
        population = elites + population_x

        # Reavalia o melhor indivíduo para registrar
        fitness_value = fitness(best_individual)
        bestValue.append(fitness_value)

        # Calcula diversidade da população atual
        std_devs, dist_to_center, pairwise_dist = diversity_metrics(population)

        # Aumento da taxa de mutação em caso de baixa diversidade
        if(MUTATION_ENHANCEMENT and pairwise_dist < MUTATION_ENHANCE_SETPOINT):
            mut_rate = MUTATION_RATE_HIGH
        else:
            mut_rate = MUTATION_RATE

        # Reinicialização da população em caso de baixa diversidade
        if(POPULATION_RESET and pairwise_dist < POPULATION_RESET_SETPOINT):
            population = elites + generate_population()
            std_devs, dist_to_center, pairwise_dist = diversity_metrics(population)

        # Armazena os valores
        std_devs_list.append(std_devs)
        mean_distance_to_center_list.append(dist_to_center)
        mean_pairwise_distance_list.append(pairwise_dist)

        # Early stopping
        if(fitness_value < fitness_for_last_iter_with_enhance):
            last_iter_with_enhancement = iter
            fitness_for_last_iter_with_enhance = fitness_value
        elif(EARLY_STOPPING and iter - last_iter_with_enhancement >= EARLY_STOPPING_PATIENCE):
            break

        if(iter_print):
            print(f"Generation: {iter} | Best Fitness: {fitness_value:.6f} | Best Solution: {best_individual}")

    return bestValue, best_individual, std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list

## 5 - Execução única - Testes

In [ ]:
fitness_plot()

In [ ]:
bestValue, best_individual, std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list = genetic_algorithm(iter_print=True)

In [ ]:
plot_diversity_metrics(std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list)

In [ ]:
x = np.arange(0, len(bestValue))
y = bestValue

fig = plt.figure()
ax = plt.gca()

plt.plot(x, y, color='Plum', linestyle='--', linewidth=2,
         marker='o', markeredgewidth=1, markerfacecolor='FireBrick',
         markeredgecolor='black', markersize=2)

plt.xlabel('Geração')
plt.ylabel('Função objetivo')
plt.xticks(np.linspace(0, len(bestValue), 8, dtype=int))

# Mostra no título qual função está sendo usada
plt.title(f'Curva de convergência - Função {FUNCTION_NAME.capitalize()}')

if(FUNCTION_NAME == "dropwave"):
    plt.axhline(y = -1, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
elif(FUNCTION_NAME == "levi"):
    plt.axhline(y = 0, color ="red", linestyle ="-", zorder=1.5, linewidth=2)

plt.grid(True)
plt.show()
plt.close(fig)

# Identificar última mudança significativa (> 1e-4) em bestValue
threshold = 1e-4
last_significant_idx = 0

for i in range(1, len(bestValue)):
    if abs(bestValue[i] - bestValue[i - 1]) > threshold:
        last_significant_idx = i

print(f"Última mudança significativa (>1e-4) ocorreu na Geração de Convergência: {last_significant_idx}")
print(f"Best Fitness : {bestValue[-1]:.6f}")
print(f"POPULATION_SIZE: {POPULATION_SIZE}")
print(f"GENERATION_COUNT: {GENERATION_COUNT}")
print(f"CROSSOVER_RATE: {CROSSOVER_RATE}")
print(f"MUTATION_RATE: {MUTATION_RATE}")
print(f"CHROMOSOME_LENGTH: {CHROMOSOME_LENGTH}")
print(f"LOWER_BOUND: {LOWER_BOUND}")
print(f"UPPER_BOUND: {UPPER_BOUND}")